# Importing the necessary modules

In [69]:
# Utilitaries
from csv import reader
import re
import pandas as pd
import zeep
# Text manipulation

# Data preparation
import nltk
from nltk.corpus import stopwords as stp
import nltk.stem.snowball as snowball
# Machine learning tools
from sklearn.model_selection import train_test_split
# Machine learning classificators

# Reading the corpus

In [2]:
df = pd.read_csv('corpus.csv')

In [3]:
df.sample(5)

,author,title,translation,label
928,TVR,"Rezultate parțiale BEC, după centralizarea a 9...","PSD a obținut la alegerile parlamentare 29,64...",1
920,TVR,Marcel Ciolacu: Voi avea o discuție cu colegii...,"Președintele PSD, Marcel Ciolacu, a declarat c...",1
430,Ionut,Cel mai norocos oltean. A găsit o sarma cu pat...,"Chiar dacă a găsit sarmaua cu patru foi, oltea...",0
700,TVR,"Bugetul, dezbătut intens în comisiile de speci...",ACTUALIZARE Bugetul Casei Naţionale de Asigură...,1
342,Eftimie,Fum alb la negocieri. Ludovic Orban fumează ia...,"„Doar un fum să mai trag”, a declarat liderul ...",0


# Data preparation

### Lemmatization
_Source: http://nlptools.info.uaic.ro/WebPosRo/_

In [4]:
wsdl = 'http://nlptools.info.uaic.ro/WebPosRo/PosTaggerRoWS?wsdl'

In [5]:
client = zeep.Client(wsdl=wsdl)

#### Transforming each article using the POS tagger,tokenizing and filtering the stopwords at the same time

In [ ]:
lemmatized_articles_xml=[]

In [6]:
for article in df['translation']:
     lemmatized_articles_xml.append(client.service.parseSentence_XML(article))
    

In [10]:
# # Saving the articles to save execution time
# for article in lemmatized_articles_xml:
#     with open('lemmatized_articles','a+') as f:
#         f.write(article)
#         f.write('\n')

In [62]:
stopwords = stp.words('romanian')
lemmatized_articles_clean=[]
for article in lemmatized_articles_xml:
    article = article.split('\n')
    lemmas = []
    for line in article:
        found = re.search('.*LEMMA="([A-Za-zăâîșț1-9]*)".*', line)
        if found:
            word = found.group(1)
            if word not in stopwords:
                lemmas.append(word)
    lemmatized_articles_clean.append(lemmas)

#### Adding the lemmatized articles to the corpus

In [63]:
df['lemmatized_articles'] = lemmatized_articles_clean

In [67]:
df.sample(5)

,author,title,translation,label,lemmatized_articles
838,TVR,Cum își petrec politicienii Crăciunul,Masa trebuie să fie plină de bucate speciale î...,1,"[masă, trebui, plin, bucată, special, Cristina..."
542,Vasile,Testul COVID făcut de naționala de fotbal arat...,"„COVID n-am găsit, dar până la urmă, mai conte...",0,"[COVID, găsi, conta, adăuga, medic, testa, spo..."
742,TVR,"Diana Șoșoacă, fără sprijinul partidului AUR","DAN TANASĂ, deputat AUR: ""M-a deranjat acel ep...",1,"[dan, tanasă, deputat, aur, deranja, episod, d..."
141,Calin,Ce ghiolban! O specie de țânțar râgâie după ce...,"”Nu sunt adeptul soluțiilor extreme, dar aceas...",0,"[adept, soluție, extrem, specie, țânțar, trebu..."
751,TVR,Orban: Nu s-a luat nicio decizie legată de spo...,El a adăugat că studenţii vor avea în continua...,1,"[adăuga, student, vrea, gratuitate, călătorie,..."


#### Stemming

In [73]:
stemmed_articles = []
stemmer = snowball.SnowballStemmer('romanian')

for article in df['lemmatized_articles']:
    stemmed_words =[]
    for word in article:
        stemmed_words.append(stemmer.stem(word))
    stemmed_articles.append(stemmed_words)

#### Adding the stemmed articles to the corpus

In [75]:
df['stemmed_articles']=stemmed_articles

In [76]:
df.sample(5)

,author,title,translation,label,lemmatized_articles,stemmed_articles
524,TNR,Președinte jucător! Iohannis s-a dus în locul ...,"„N-am mai fost în Dubai de ani buni, darămite ...",0,"[Dubai, an, bun, darămite, juca, tenis, recuno...","[duba, an, bun, darăm, juc, tenis, recunoașt, ..."
290,Calin,"Marian Godină, în carantină pentru două săptăm...",Marian Godină spune că va profita de perioada ...,0,"[Marian, godină, spune, vrea, profita, perioad...","[marian, godin, spun, vre, profit, perioad, sc..."
874,TVR,"Ciolacu, scrisoare către Iohannis: Vă cer să f...",El spune că este „un abuz intolerabil comis de...,1,"[spune, abuz, intolerabil, comis, partid, sanc...","[spun, abuz, intoler, comis, partid, sancțion,..."
685,TVR,Președintele Iohannis le-a transmis o urare de...,Şeful statului le-a transmis o urare doamnelor...,1,"[șef, stat, transmite, urare, doamnă, aștepta,...","[șef, stat, transmit, urar, doamn, aștept, ieș..."
816,TVR,"Costel Alexe, așteptat la DNA. Fostul demnitar...",Fostul ministru al mediului Costel Alexe e acu...,1,"[ministru, mediu, costel, alexe, acuzat, doamn...","[ministru, mediu, costel, alex, acuz, doamn, p..."
